In [ ]:
%pip install geopy

In [5]:
from geopy.geocoders import Nominatim

# Инициализация геокодера
geolocator = Nominatim(user_agent="geoapi")

# Получение координат центра Москвы
location = geolocator.geocode("Юрга, Россия")
print(f"Координаты центра: {location.latitude}, {location.longitude}")

# Полный ответ с деталями
print(location.raw) 

Координаты центра: 55.7136, 84.934799
{'place_id': 201975429, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 3396612, 'lat': '55.7136000', 'lon': '84.9347990', 'class': 'place', 'type': 'town', 'place_rank': 18, 'importance': 0.5214094955471994, 'addresstype': 'town', 'name': 'Юрга', 'display_name': 'Юрга, Юргинский городской округ, Кемеровская область, Сибирский федеральный округ, Россия', 'boundingbox': ['55.6743708', '55.7656786', '84.7988863', '85.0163092']}
